In [1]:
import pandas as pd
import torch
from torch_rechub.models.multi_task import SharedBottom, MMOE, PLE, AITM, fus_moe_add
from torch_rechub.trainers import MTLTrainer
from torch_rechub.basic.features import DenseFeature, SparseFeature
from torch_rechub.utils.data import DataGenerator
from torch_rechub.single2fus_ali import single2fus_ali
from torch_rechub.models.multi_task.mtl_lib import AdaTTSp

In [ ]:
model_name = 'AdaTTSp' # select the model you want to train, 'fus_moe_add' is our method
epoch = 10
learning_rate = 1e-3
batch_size = 32768
weight_decay = 1e-5
save_dir = './save_dir'
seed = 2024
gpus = None
torch.manual_seed(seed)

In [ ]:
device = 'cpu'
if torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

In [4]:
def get_ali_ccp_data_dict(model_name, data_path="data/ali-ccp"):
    df_train = pd.read_csv(data_path + '/ali_ccp_train.csv')
    df_val = pd.read_csv(data_path + '/ali_ccp_val.csv')
    df_test = pd.read_csv(data_path + '/ali_ccp_test.csv')
    print("train : val : test = %d %d %d" % (len(df_train), len(df_val), len(df_test)))
    train_idx, val_idx = df_train.shape[0], df_train.shape[0] + df_val.shape[0]
    data = pd.concat([df_train, df_val, df_test], axis=0)
    #task 1 (as cvr): main task, purchase prediction
    #task 2(as ctr): auxiliary task, click prediction
    data.rename(columns={'purchase': 'cvr_label', 'click': 'ctr_label'}, inplace=True)
    data["ctcvr_label"] = data['cvr_label'] * data['ctr_label']

    col_names = data.columns.values.tolist()
    dense_cols = ['D109_14', 'D110_14', 'D127_14', 'D150_14', 'D508', 'D509', 'D702', 'D853']
    sparse_cols = [col for col in col_names if col not in dense_cols and col not in ['cvr_label', 'ctr_label', 'ctcvr_label']]
    print("sparse cols:%d dense cols:%d" % (len(sparse_cols), len(dense_cols)))
    #define dense and sparse features
    label_cols = ['cvr_label', 'ctr_label']  #the order of labels can be any
    used_cols = sparse_cols + dense_cols
    features = [SparseFeature(col, data[col].max()+1, embed_dim=4)for col in sparse_cols] \
                + [DenseFeature(col) for col in dense_cols]
    x_train, y_train = {name: data[name].values[:train_idx] for name in used_cols}, data[label_cols].values[:train_idx]
    x_val, y_val = {name: data[name].values[train_idx:val_idx] for name in used_cols}, data[label_cols].values[train_idx:val_idx]
    x_test, y_test = {name: data[name].values[val_idx:] for name in used_cols}, data[label_cols].values[val_idx:]
    return features, x_train, y_train, x_val, y_val, x_test, y_test

In [ ]:
# Set model hyperparameters
if model_name == "SharedBottom":
    features, x_train, y_train, x_val, y_val, x_test, y_test = get_ali_ccp_data_dict(model_name)
    task_types = ["classification", "classification"]
    model = SharedBottom(features, task_types, bottom_params={"dims": [128,64]}, tower_params_list=[{"dims": [32]}, {"dims": [32]}])
elif model_name == "MMOE":
    features, x_train, y_train, x_val, y_val, x_test, y_test = get_ali_ccp_data_dict(model_name)
    task_types = ["classification", "classification"]
    model = MMOE(features, task_types, 12, expert_params={"dims": [128,64]}, tower_params_list=[{"dims": [32]}, {"dims": [32]}])
elif model_name == "PLE":
    features, x_train, y_train, x_val, y_val, x_test, y_test = get_ali_ccp_data_dict(model_name)
    task_types = ["classification", "classification"]
    model = PLE(features, task_types, n_level=2, n_expert_specific=2, n_expert_shared=2, expert_params={"dims": [128,64]}, tower_params_list=[{"dims": [32]}, {"dims": [32]}])
elif model_name == "AITM":
    task_types = ["classification", "classification"]
    features, x_train, y_train, x_val, y_val, x_test, y_test = get_ali_ccp_data_dict(model_name)
    model = AITM(features, 2, bottom_params={"dims": [128,64]}, tower_params_list=[{"dims": [32]}, {"dims": [32]}])
elif model_name == "AdaTTSp":
    task_types = ["classification", "classification"]
    features, x_train, y_train, x_val, y_val, x_test, y_test = get_ali_ccp_data_dict(model_name)
    model = AdaTTSp(input_dim=features, expert_out_dims=[[128,64]], num_tasks=2, num_task_experts=12)
elif model_name == "fus_moe_add":
    task_types = ["classification", "classification"]
    features, x_train, y_train, x_val, y_val, x_test, y_test = get_ali_ccp_data_dict(model_name)
    model = fus_moe_add(features, task_types, n_level=3, n_expert_specific=2, expert_params1 ={"dims": [128,64]}, expert_params2 ={"dims": [256,64]}, tower_params_list=[{"dims": [32]}, {"dims": [32]}])

In [6]:
if model_name == "fus_moe_add":
    model_path = "path/to/save_dir/single_model_name_1.pth"
    single_list = ['mlp1', 'mlp2', 'mlp3']
    fus_list = ['cgc_layers.0.experts_specific.0','cgc_layers.1.experts_specific.0', 'cgc_layers.2.experts_specific.0']
    single2fus_ali(model_path, single_list, fus_list, model)

    model_path = "path/to/save_dir/single_model_name_2.pth"
    single_list = ['mlp1', 'mlp2', 'mlp3']
    fus_list = ['cgc_layers.0.experts_specific.2','cgc_layers.1.experts_specific.2', 'cgc_layers.2.experts_specific.2']
    single2fus_ali(model_path, single_list, fus_list, model)

    model_path = "path/to/save_dir/single_model_name_3.pth"
    single_list = ['mlp1', 'mlp2', 'mlp3']
    fus_list = ['cgc_layers.0.experts_specific.1','cgc_layers.1.experts_specific.1', 'cgc_layers.2.experts_specific.1']
    single2fus_ali(model_path, single_list, fus_list, model)

    model_path = "path/to/save_dir/single_model_name_4.pth"
    single_list = ['mlp1', 'mlp2', 'mlp3']
    fus_list = ['cgc_layers.0.experts_specific.3','cgc_layers.1.experts_specific.3', 'cgc_layers.2.experts_specific.3']
    single2fus_ali(model_path, single_list, fus_list, model)

In [8]:
dg = DataGenerator(x_train, y_train)
train_dataloader, val_dataloader, test_dataloader = dg.generate_dataloader(x_val=x_val, y_val=y_val, x_test=x_test, y_test=y_test, batch_size=batch_size)
mtl_trainer = MTLTrainer(model, task_types=task_types, optimizer_params={"lr": learning_rate, "weight_decay": weight_decay}, n_epoch=epoch, earlystop_patience=5, device=device, gpus=gpus, model_path=save_dir)
# started training
file_path = 'path/to/save/output/result_{}_{}.txt'.format(model_name)
mtl_trainer.fit(train_dataloader, val_dataloader, mode = '{}'.format(model_name), seed = '', file_path = file_path)

In [ ]:
auc = mtl_trainer.evaluate(mtl_trainer.model, test_dataloader)
print(f'test auc: {auc}')

epo = ['test_result']
my_list = epo + auc
my_list = ', '.join(map(str, my_list))

try:
    with open(file_path, 'a') as file:
        file.write(my_list + '\n')
except Exception as e:  
    print(f"An error occurred while adding to the file: {e}")